<h1>Mathematical Theory of Data Assimilation with Applications:<br>

<p class="fragment">Tutorial part 4 of 4 --- Bayesian DA through sampling<p></h1>


<h3>Stochastic EnKF in the Ikeda map</h3>

<ul>
    <li class="fragment"><b>Exercise (3 minutes):</b>We will now examine the performance of the EnKF, using once again in the Ikeda map in a twin experiment.</li>
    <li class="fragment">In the following code, we will plot the forecast and analysis <em>ensembles</em> to demonstrate how they track the observations.</li>
    <li class="fragment">The mean of each ensemble will be plotted as a diamond, while ensemble members will be plotted as opaque points.</li>
</ul>

<h3>Stochastic EnKF in the Ikeda map</h3>

<ul>
    <li class="fragment">We want to evaluate the following questions:</li>
    <ol>
        <li class="fragment">How does the performance of the EnKF change with the number of samples?</li>
        <li class="fragment">How does the performance of the EnKF change over the number of assimilation steps?</li>
        <li class="fragment">How does the performance of the EnKF change with respect to the initial prior uncertainty $B_{var}$?</li>
        <li class="fragment">How does the performance of the EnKF change with respect to the observational error variance $R_{var}$?</li>
    </ol>
</ul>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
from matplotlib.patches import Ellipse

def Ikeda(X_0, u):
    """The array X_0 will define the initial condition and the parameter u controls the chaos of the map
    
    This should return X_1 as the forward state."""
    
    t_n = 0.4 - 6 / (1 + X_0.dot(X_0) )
    
    x_1 = 1 + u * (X_0[0] * np.cos(t_n) + X_0[1] * np.cos(t_n))
    y_1 = u * (X_0[0] * np.sin(t_n) + X_0[1] * np.cos(t_n))
                 
    X_1 = np.array([x_1, y_1])
    
    return X_1

def Ikeda_V(X_0, u):
    """The array X_0 will define the ensemble matrix of dimension 2 times N_ens
    
    This should return X_1 as the forward state."""
    
    t_n = 0.4 - 6 / (1 + np.sum(X_0*X_0, axis=0) )
    
    x_1 = 1 + u * (X_0[0, :] * np.cos(t_n) + X_0[1, :] * np.cos(t_n))
    y_1 = u * (X_0[0, :] * np.sin(t_n) + X_0[1, :] * np.cos(t_n))
                 
    X_1 = np.array([x_1, y_1])
    
    return X_1

In [ ]:
def animate_enkf(B_var = 0.1, R_var = 0.1, N=2, ens_n=3):

    # define the static background and observational error covariances
    P_0 = B_var * np.eye(2)
    R = R_var * np.eye(2)

    # set a random seed for the reproducibility
    np.random.seed(1)
    
    # we define the mean for the background
    x_b = np.array([0,0])
    
    # and the initial condition of the real state as a random draw from the prior
    x_t = np.random.multivariate_normal([0,0], P_0)

    y_obs = np.zeros([2,N-1])
    
    # define the Ikeda map parameter
    u = 0.9
    for i in range(N-1):
        # we forward propagate the true state
        x_t = Ikeda(x_t, u)
    
        # and generate a noisy observation
        y_obs[:, i] = x_t + np.random.multivariate_normal([0,0], R)
    
    
    # we define the ensemble as a random draw of the prior
    ens = np.random.multivariate_normal(x_b, P_0, size=ens_n).transpose()

    
    # define the Ikeda map parameter
    for i in range(N-1):
        
        # forward propagate the last analysis
        ens_f = Ikeda_V(ens, u)
        
        # we generate observation perturbations
        obs_perts =  np.random.multivariate_normal([0,0], R, size=ens_n)
        obs_perts = obs_perts - np.mean(obs_perts, axis=0)
        
        # we generate the ensemble based observation error covariance 
        obs_cov = obs_perts.transpose() @ obs_perts / (ens_n - 1)
        
        # we perturb the observations
        perts_obs = np.squeeze(y_obs[:,i]) + obs_perts
        
        # we compute the ensemble mean and anomalies
        X_mean_f = np.mean(ens_f, axis=1)
        A_t = (ens_f.transpose() - X_mean_f) / np.sqrt(ens_n - 1)
        
        # and the ensemble covariances
        P = A_t.transpose() @ A_t

        # we compute the ensemble based gain and the analysis ensemble
        K_gain = P @ np.linalg.inv( P + obs_cov)
        ens = ens_f + K_gain @ (perts_obs.transpose() - ens_f)
        X_mean_a = np.mean(ens, axis=1)

    
    fig = plt.figure(figsize=(16,8))
    ax = fig.add_axes([.1, .1, .8, .8])
    
    l1 = ax.scatter(ens_f[0, :], ens_f[1, :], c='k', s=20, alpha=.5, marker=',')
    ax.scatter(X_mean_f[0], X_mean_f[1], c='k', s=200, marker="D")
    
    l3 = ax.scatter(ens[0, :], ens[1, :], c='b', s=20, alpha=.5, marker=',')
    ax.scatter(X_mean_a[0], X_mean_a[1], c='b', s=200, marker="D")
    
    l2 = ax.scatter(y_obs[0, -1], y_obs[1, -1], c='r', s=20)
    ax.add_patch(Ellipse(y_obs[:,-1], R_var, R_var, ec='r', fc='none'))
    
    
    ax.set_xlim([-2, 4])
    ax.set_ylim([-4,2])
    
    labels = ['Forecast', 'Observation', 'Analysis']
    plt.legend([l1,l2,l3],labels, loc='upper right', fontsize=26)
    plt.show()
    
w = interactive(animate_enkf,B_var=(0.01,1.0,0.01), R_var=(0.01,1.0,0.01), N=(2, 50, 1), ens_n=(3,300, 3))
display(w)

<h3>Summary of the EnKF</h3>

<ul>
    <li class="fragment">The EnKF makes a vast improvement over the earlier explored methods and demonstrates its robustness as a learning scheme <em>when there are sufficiently many samples.</em></li>
    <ul>
        <li class="fragment">It should be noted that this requires vastly fewer samples than implementing an effective bootstrap particle filter, but at the cost of introducing bias in the analysis of the posterior.</li>
    </ul>
    <li class="fragment">However in operational settings, the reality is that ensemble-based forecasting is still highly expensive and most operational EnKF uses at most $\mathcal{O}\left(10^2\right)$ samples in the learning.</li>
    <li class="fragment">While the EnKF is highly parallelizable, numerical weather prediciton models require massive computation power and this fundamentally limits the number of available samples.</li>
</ul>

<h3>Making the EnKF work in practice</h3>

<ul>
    <li class="fragment">The reality of implementing the EnKF in a numerical weather prediction setting is that the covariance estimates will also be highly biased and extremely rank deficient.</li>
    <li class="fragment">While there are reasons to belive that the true Bayesian posterior covariance is also rank deficient, in the end we especially rely on:</li>
    <ol>
        <li class="fragment">inflation; and</li>
        <li class="fragment">localization;</li>
    </ol>
    <li class="fragment">in order to relax the error estimates (introduce variance) and rectify the extreme rank deficiency.</li>
    <li class="fragment">Both of these techniques are highly active research areas for improving ensemble-based filtering, which are discussed in, e.g., the recent review article of Carrassi et al.</li>
</ul>